<a href="https://colab.research.google.com/github/shaunakg/gpt2-simple-social/blob/master/GPT_2_Text_Generation_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

Original notebook by [Max Woolf](http://minimaxir.com)  
Updated for social media scraping by [Shaunak G](https://github.com/shaunakg)  

Retrain an advanced text generating neural network on any text dataset for free on a GPU using Collaboratory using `gpt-2-simple`. Scrape text from reddit, twitter or tumblr using the associated libraries and train the neural network with it.

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read the original creator's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook.

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below.  
  
*Note: some cells have been commented out for continuity when running all (Runtime -> Restart and Run All)*


## Download posts from social media using API

In [0]:
!pip install -q python-dotenv pytumblr tweepy praw

In [2]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

# Using .txt not .env because it allows editing in Google Colab
dotenv_path = 'config.txt'
load_dotenv(dotenv_path)

social_media = os.environ.get("social_platform")# Supported: twitter, tumblr as string
social_user = os.environ.get("social_user") # User handle. Currently not supporting custom blog domains. Do not include @ or .tumblr.com. For reddit, this will fetch subreddit titles. Do not include /r/

training_file_name = social_media + "_" + social_user + "_training.txt"

if social_media == "tumblr":

  import pytumblr

  # Authenticate via OAuth
  client = pytumblr.TumblrRestClient(
    os.environ.get('tumblr_auth_1'),
    os.environ.get('tumblr_auth_2'),
    os.environ.get('tumblr_auth_3'),
    os.environ.get('tumblr_auth_4')
  )

  blogposts = client.posts(social_user + ".tumblr.com", type='text', reblog_info='false', filter='text', tag='dream')

  with open(__tumblr_file_name, "a+") as writefile:
    for i in blogposts['posts']:
      writefile.write(i['body'] + '\n')

elif social_media == "twitter":

  import tweepy

  twitter_auth = tweepy.OAuthHandler(os.environ.get('twitter_consumer_key'), os.environ.get('twitter_consumer_secret'))
  twitter_auth.set_access_token(os.environ.get('twitter_at_key'), os.environ.get('twitter_at_secret'))
  print("Initialised twitter API")

  twitter_api = tweepy.API(twitter_auth)

  with open(training_file_name, "a+") as writefile:
    twitter_tweets = twitter_api.user_timeline(screen_name = social_user,count=200)

    for i in twitter_tweets:
      writefile.write( i.text + "\n" )

  print("Written all posts on " + social_media + " by " + social_user + " to " + training_file_name)

elif social_media == "reddit":
  import praw

  reddit = praw.Reddit(client_id=os.environ.get("reddit_client_id"),
                      client_secret=os.environ.get("reddit_client_secret"),
                      user_agent=os.environ.get("reddit_user_agent"))

  with open(training_file_name, "a+") as writefile:
    for submission in reddit.subreddit(social_user).top("all", limit=100):
        writefile.write(submission.title + "\n")

  print("Written the 100 top post titles in reddit's r/" + social_user + " to " + training_file_name)

else:
  exit("Unsupported or unset social media platform.")

Written the 100 top post titles in reddit's r/tifu to reddit_tifu_training.txt


## Do the main NN learning and generation

In [3]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

# Setup variables from config file
ml_model_name = os.environ.get("model_name") # Usually 124M for mix between accuracy and speed
ml_training_steps = int(os.environ.get("training_steps")) # Usually 100
ml_generation_length = int(os.environ.get("generation_length")) # Usually 1000
ml_temp = float(os.environ.get("generation_temperature")) # How 'crazy' the output is. Recommend between 0.7-1
ml_samples = int(os.environ.get("generation_samples")) # Usually 100

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [4]:
!nvidia-smi

Thu Apr 23 01:43:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [5]:
gpt2.download_gpt2(model_name=ml_model_name)

Fetching checkpoint: 1.05Mit [00:00, 183Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 102Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 223Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 78.6Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 211Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 146Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 120Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [6]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = training_file_name

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
# Don't use Google Drive ATM
# gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [9]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=ml_model_name,
              steps=ml_training_steps,
              restore_from='fresh',
              run_name='run1',
              print_every=1,
              sample_every=5,
              save_every=10
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 479.18it/s]

Loading dataset...
dataset has 1729 tokens
Training...


[1 | 8.68] loss=3.19 avg=3.19
[2 | 9.93] loss=2.92 avg=3.06
[3 | 11.19] loss=2.59 avg=2.90
[4 | 12.44] loss=2.37 avg=2.77
[5 | 13.69] loss=2.10 avg=2.63
======== SAMPLE 1 ========

I did not know what I thought.
D.C. United (USA)
D.C. United (USA)
D.C. United (USA)
I did not know what I thought.
I thought I had a headache.
D.C. United (USA)
D.C. United (USA)
D.C. United (USA)
I did not know what I thought.
I thought I had a headache.
I thought I had a head injury.
D.C. United (USA)
D.C. United (USA)
I did not know what I thought.
I thought on the bus that the plane was heading to Amsterdam.
D.C. United (UK)
D.C. United (UK)
D.C. United (UK)
I had to take to the ER for chest compressions.
D.C. United (UK)
D.C. United (UK)
I did not know what I thought.
D.C. United (UK)
D.C. United (UK)
I felt dizzy. (sadly, not to use my real name here)
D.C. United (GB)
D.C. United (GB)
D.C. United (GB)
I knew something was not right with my phone.
I was looking through my purse when I saw something.
I 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
# Don't use Google Drive ATM
# gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
# These cells interrupt a full run.
#gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
# These cells interrupt a full run.
#sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
#gpt2.generate(sess, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
# gpt2.generate(sess,
#               length=ml_generation_length,
#               temperature=ml_temp,
#               nsamples=1,
#               batch_size=1
#               )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_" + social_media + "_" + social_user + "_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=ml_generation_length,
                      temperature=ml_temp,
                      nsamples=ml_samples,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
# model_name = "774M"
# gpt2.download_gpt2(model_name=model_name)

In [0]:
# sess = gpt2.start_tf_sess()

# gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
# gpt2.generate(sess,
#               model_name=model_name,
#               prefix="The secret of life is",
#               length=100,
#               temperature=0.7,
#               top_p=0.9,
#               nsamples=5,
#               batch_size=5
#               )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
# !kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.